<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Analyse_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dans ce carnet nous allons mettre en application les outils de base d'analyse et de traitement des séries temporelles :
  - Autocorrélation 
  - Autocorrélation partielle
  - Suppression d'une tendance linéaire d'une série
  - Différenciation non saisonnale et saisonnale
  - Suppression d'une tendance non linéaire d'une série
  - Supression d'une saisonnalité
  - Test de Dickey-Fuller (test de stationnarité)

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
!pip install statsmodels --upgrade

# Chargement des séries

In [ ]:
# Fonction permettant d'afficher une série temporelle
def affiche_serie(temps, serie, format="-", debut=0, fin=None, label=None):
    plt.plot(temps[debut:fin], serie[debut:fin], format, label=label)
    plt.xlabel("Temps")
    plt.ylabel("Valeur")
    if label:
        plt.legend(fontsize=14)
    plt.grid(True)

!rm *.csv
!rm *.csv.*
!wget -q --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/data/serie_hetero.csv"
!wget -q --no-check-certificate --content-disposition  "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/data/serie_homo.csv"
!wget -q --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/data/serie_bb.csv"
!wget -q --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/data/serie_rw.csv"
!wget -q --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/data/serie_bb_trend.csv"
!wget -q --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/data/GDP.csv"
!wget -q --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/data/Data_Airline.csv"
!wget -q --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/data/Data_Accidental.csv"


serie_hetero = pd.read_csv("serie_hetero.csv", usecols=[1], engine='python')
serie_homo = pd.read_csv("serie_homo.csv", usecols=[1], engine='python')
serie_bb = pd.read_csv("serie_bb.csv", usecols=[1], engine='python')
serie_rw = pd.read_csv("serie_rw.csv", usecols=[1], engine='python')
serie_bb_trend = pd.read_csv("serie_bb_trend.csv", usecols=[1], engine='python')
serie_gdp = pd.read_csv("GDP.csv", usecols=[0], engine='python')
serie_data_airline = pd.read_csv("Data_Airline.csv", usecols=[0], engine='python')
serie_data_accidental = pd.read_csv("Data_Accidental.csv", usecols=[0], engine='python')


serie_hetero = np.asarray(serie_hetero.astype('float32'))[:,0]
serie_homo = np.asarray(serie_homo.astype('float32'))[:,0]
serie_bb = np.asarray(serie_bb.astype('float32'))[:,0]
serie_rw = np.asarray(serie_rw.astype('float32'))[:,0]
serie_bb_trend = np.asarray(serie_bb_trend.astype('float32'))[:,0]
serie_gdp = np.asarray(serie_gdp.astype('float32'))[:,0]
serie_data_airline = np.asarray(serie_data_airline.astype('float32'))[:,0]
serie_data_accidental = np.asarray(serie_data_accidental.astype('float32'))[:,0]

In [ ]:
serie_gdp = pd.read_csv("GDP.csv", usecols=[0], engine='python')

In [ ]:
serie_gdp = np.asarray(serie_gdp.astype('float32'))[:,0]

In [ ]:
serie_gdp

#1. Bruit blanc




In [ ]:
temps = np.linspace(0,len(serie_bb),len(serie_bb))
plt.figure(figsize=(10, 6))
affiche_serie(temps,serie_bb)
plt.title('Bruit blanc')
plt.show()

In [ ]:
# ACF & PACF du bruit blanc

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_bb, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du bruit blanc")

plot_pacf(serie_bb, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle du bruit blanc")


Test de Dickey-Fuller

In [ ]:
import statsmodels.api as sm

serie_test = serie_bb

adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

#2. Bruit blanc avec tendance

Voyons comment supprimer la tendance sur un bruit blanc

In [ ]:
temps = np.linspace(0,len(serie_bb),len(serie_bb_trend))
plt.figure(figsize=(10, 6))
affiche_serie(temps,serie_bb_trend)
plt.title('Bruit blanc avec tendance')
plt.show()

In [ ]:
# ACF & PACF du bruit blanc avec tendance

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_bb_trend, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du bruit blanc avec tendance")

plot_pacf(serie_bb_trend, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle du bruit blanc avec tendance")

In [ ]:
# Detendring du bruit blanc avec tendance
from scipy import signal

serie_bb_detrend = signal.detrend(serie_bb_trend)

plt.figure(figsize=(10, 6))
affiche_serie(temps,serie_bb_detrend)
plt.title('Signal sans tendance')
plt.show()

In [ ]:
# ACF & PACF du signal

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_bb_detrend, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du bruit blanc avec tendance")

plot_pacf(serie_bb_detrend, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle du bruit blanc avec tendance")

In [ ]:
import statsmodels.api as sm

serie_test = serie_bb_trend
#serie_test = serie_bb_detrend

adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

#3. Random Walk

Voyons comment supprimer la tendance et différencier un signal à l'odre 1

In [ ]:
temps = np.linspace(0,len(serie_bb),len(serie_rw))

plt.figure(figsize=(10, 6))
affiche_serie(temps,serie_rw)
plt.title('Random Walk')
plt.show()

In [ ]:
# ACF & PACF du random walk

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_rw, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du random walk")

plot_pacf(serie_rw, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle du random walk")

In [ ]:
# Detendring du random walk
from scipy import signal

serie_rw_detrend = signal.detrend(serie_rw)

plt.figure(figsize=(10, 6))
affiche_serie(temps,serie_rw_detrend)
plt.title('Signal sans tendance')
plt.show()

In [ ]:
# ACF & PACF du signal

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_rw_detrend, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du signal sans tendance")

plot_pacf(serie_rw_detrend, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation du signal sans tendance")

In [ ]:
# Différenciation d'ordre 1

serie_rw_detrend_dif1 = serie_rw_detrend[1:] - serie_rw_detrend[:-1]
serie_rw_dif1 = serie_rw[1:] - serie_rw[:-1]


In [ ]:
f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

ax1.plot(temps[1:],serie_rw_detrend_dif1)
ax1.set_title("Signal sans tendance différencié #1")

ax2.plot(temps[1:],serie_rw_dif1)
ax2.set_title("Signal différencié #1")

In [ ]:
# ACF & PACF du signal

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_rw_detrend_dif1, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du signal sans tendance différencié #1")

plot_pacf(serie_rw_detrend_dif1, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle du signal sans tendance différencié #1")

plot_acf(serie_rw_dif1, ax=ax3, lags = range(0,50))
ax3.set_title("Autocorrélation du signal différencié #1")

plot_pacf(serie_rw_dif1, ax=ax4, lags = range(0, 50))
ax4.set_title("Autocorrélation partielle du signal différencié #1")

In [ ]:
import statsmodels.api as sm

serie_test = serie_rw
#serie_test = serie_rw_detrend
#serie_test = serie_rw_detrend_dif1


adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

#4. Série Homoscédatique

In [ ]:
temps = np.linspace(0,len(serie_bb),len(serie_homo))

plt.figure(figsize=(10, 6))
affiche_serie(temps,serie_homo)
plt.title('Série temporelle homoscédatique')
plt.show()

In [ ]:
# ACF & PACF de la série Homoscédatique

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_homo, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation de la série Homoscédatique")

plot_pacf(serie_homo, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle de la série Homoscédatique")

In [ ]:
# Detendring de la série homoscédatique
from scipy import signal

serie_homo_detrend = signal.detrend(serie_homo)

plt.figure(figsize=(10, 6))
affiche_serie(temps,serie_homo_detrend)
plt.title('Signal sans tendance')
plt.show()

In [ ]:
# ACF & PACF du signal

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_homo_detrend, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du signal")

plot_pacf(serie_homo_detrend, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation du signal")

In [ ]:
# Différenciation d'ordre 1

from statsmodels.tsa.statespace.tools import diff

serie_homo_detrend_dif1 = diff(serie_homo_detrend,1)

plt.figure(figsize=(10, 6))
affiche_serie(temps[1:],serie_homo_detrend_dif1)
plt.title("Signal différencié d'ordre 1")
plt.show()

In [ ]:
# ACF & PACF du signal

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_homo_detrend_dif1, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du signal après diff #1")

plot_pacf(serie_homo_detrend_dif1, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation du signal après diff #1")

In [ ]:
# Différenciation d'odre 1 et saisonnale à l'odre 1 et de période 365

from statsmodels.tsa.statespace.tools import diff

serie_homo_detrend_sdif1 = diff(serie_homo,1,1,365)       # diff=1 ; diff_saison=1 ; periode = 365

lag = len(temps) - len(serie_homo_detrend_sdif1)

plt.figure(figsize=(10, 6))
affiche_serie(temps[lag:],serie_homo_detrend_sdif1)
plt.title("Signal différencié d'ordre 1 + saisonalité")
plt.show()

In [ ]:
# ACF & PACF du signal

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_homo_detrend_sdif1, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du signal")

plot_pacf(serie_homo_detrend_sdif1, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle du signal")

In [ ]:
import statsmodels.api as sm

serie_test = serie_homo
serie_test = serie_homo_detrend
serie_test = serie_homo_detrend_dif1
serie_test = serie_homo_detrend_sdif1


adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

#5. Série Hétéroscédatique

In [ ]:
temps = np.linspace(0,len(serie_bb),len(serie_hetero))

plt.figure(figsize=(10, 6))
affiche_serie(temps,serie_hetero)
plt.title('Série temporelle hétéroscédatique')
plt.show()

In [ ]:
# ACF & PACF de la série Hétéroscédique

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_hetero, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation de la série hétéroscédique")

plot_pacf(serie_hetero, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation de la série hétéroscédique")

In [ ]:
# Detendring de la série hétéroscédique
from scipy import signal

serie_hetero_detrend = signal.detrend(serie_hetero)

plt.figure(figsize=(10, 6))
affiche_serie(temps,serie_hetero_detrend)
plt.title('Signal sans tendance')
plt.show()

In [ ]:
# Différenciation d'ordre 1

from statsmodels.tsa.statespace.tools import diff

serie_hetero_detrend_dif1 = diff(serie_hetero_detrend,1)

plt.figure(figsize=(10, 6))
affiche_serie(temps[1:],serie_hetero_detrend_dif1)
plt.title("Signal différencié d'ordre 1")
plt.show()

In [ ]:
# ACF & PACF du signal

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_hetero_detrend_dif1, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du signal")

plot_pacf(serie_hetero_detrend_dif1, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation du signal")

In [ ]:
# Différenciation d'odre 1 et saisonnale à l'odre 1 et de période 365

from statsmodels.tsa.statespace.tools import diff

serie_hetero_detrend_sdif1 = diff(serie_hetero,1,1,365)       # diff=1 ; diff_saison=1 ; periode = 365

lag = len(temps) - len(serie_hetero_detrend_sdif1)

plt.figure(figsize=(10, 6))
affiche_serie(temps[lag:],serie_hetero_detrend_sdif1)
plt.title("Signal différencié d'ordre 1 + saisonalité")
plt.show()

In [ ]:
# ACF & PACF du signal

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_hetero_detrend_dif1, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du signal")

plot_pacf(serie_hetero_detrend_dif1, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation du signal")

In [ ]:
import statsmodels.api as sm

serie_test = serie_hetero
serie_test = serie_hetero_detrend
serie_test = serie_hetero_detrend_dif1
#serie_test = serie_hetero_detrend_sdif1


adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

#6. PIB Américain 1947-2005

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(serie_gdp)
plt.title('Série GDP')
plt.show()

In [ ]:
# ACF & PACF de la série

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_gdp, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation de la série")

plot_pacf(serie_gdp, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation de la série")

In [ ]:
# Différenciation d'ordre 1

from statsmodels.tsa.statespace.tools import diff

serie_gdp_diff1 = diff(serie_gdp,1)

plt.figure(figsize=(10, 6))
plt.plot(serie_gdp_diff1)
plt.title("Signal différencié d'ordre 1")
plt.show()

In [ ]:
# ACF & PACF de la série

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_gdp_diff1, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation de la série")

plot_pacf(serie_gdp_diff1, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation de la série")

In [ ]:
# Différenciation d'ordre 2

from statsmodels.tsa.statespace.tools import diff

serie_gdp_diff2 = diff(serie_gdp,2)

plt.figure(figsize=(10, 6))
plt.plot(serie_gdp_diff2)
plt.title("Signal différencié d'ordre 2")
plt.show()

In [ ]:
# ACF & PACF de la série

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_gdp_diff2, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation de la série différenciée #2")

plot_pacf(serie_gdp_diff2, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle de la série différencié #2")

In [ ]:
import statsmodels.api as sm

serie_test = serie_gdp
serie_test = serie_gdp_diff1
serie_test = serie_gdp_diff2


adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

#7. Data Airline - Différenciation non saisonnale et saisonnale

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(serie_data_airline)
plt.title('Série Data Airline')
plt.show()

In [ ]:
# ACF & PACF de la série

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_data_airline, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation de la série")

plot_pacf(serie_data_airline, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle de la série")

In [ ]:
# Différenciation d'odre 1 et saisonnale à l'odre 1 et de période 12

from statsmodels.tsa.statespace.tools import diff

serie_data_airline_sdiff1 = diff(serie_data_airline,1,1,12)       # diff=1 ; diff_saison=1 ; periode = 12


plt.figure(figsize=(10, 6))
plt.plot(serie_data_airline_sdiff1)
plt.title("Signal différencié d'ordre 1 + saisonalité")
plt.show()

In [ ]:
# ACF & PACF de la série

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_data_airline_sdiff1, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation de la série")

plot_pacf(serie_data_airline_sdiff1, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle de la série")

In [ ]:
import statsmodels.api as sm

serie_test = serie_data_airline
serie_test = serie_data_airline_sdiff1


adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

#8. Accidents - Extraction de la saisonnalité

**1. Affichage de la série : Morts par accidents 1973-1978 aux Etats Unis**

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(serie_data_accidental)
plt.title('Série Data Accidental')
plt.show()

**2. ACF et PACF**

In [ ]:
# ACF & PACF de la série

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_data_accidental, ax=ax1, lags = range(0,30))
ax1.set_title("Autocorrélation de la série")

plot_pacf(serie_data_accidental, ax=ax2, lags = range(0, 30))
ax2.set_title("Autocorrélation partielle de la série")

**3. Tendance quadratique**

L'objectif est ici de trouver le polynome : P = ax4+bx3+cx2+dx+c

In [ ]:
f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Calcul des coefficients
x = np.linspace(0,len(serie_data_accidental),len(serie_data_accidental))
coefs = np.polyfit(x,serie_data_accidental,4)

# Calcul de la tendance non linéaire
trend = coefs[0]*np.power(x,4) + coefs[1]*np.power(x,3) + coefs[2]*np.power(x,2) + coefs[3]*np.power(x,1) + coefs[4]

# Calcul de la série sans tendance
serie_data_accidental_detrend = serie_data_accidental - trend

# Affiche les résultats
ax1.plot(trend)
ax1.plot(serie_data_accidental)
ax1.set_title("Série originale et tendance non linéaire")

ax2.plot(serie_data_accidental_detrend)
ax2.set_title("Série avec tendance non linéaire supprimée")

**4. Analyse de la saisonnalité**

In [ ]:
from scipy.fft import fft, fftfreq

f1, ax1 = plt.subplots(1, 1, figsize=(10, 5))

# Calcul de la FFT de la série
N = len(serie_data_accidental_detrend)
T = 1.0

fft=fft(serie_data_accidental_detrend)[:np.asarray(N/2).astype(int)]
freqs = fftfreq(N,T)[:np.asarray(N/2).astype(int)]

# Affichage
ax1.plot(freqs,abs(fft))
ax1.set_title("FFT")

In [ ]:
from scipy.signal import find_peaks

f1, ax1 = plt.subplots(1, 1, figsize=(10, 5))

peaks, _ = find_peaks(abs(fft), height=(10000))

print("Fréquences : %s" %(1.0/freqs[peaks]))

# Affichage
ax1.plot(freqs,abs(fft))
ax1.plot(freqs[peaks],abs(fft[peaks]),'x')
ax1.set_title("FFT")

In [ ]:
from statsmodels.tsa.seasonal import STL

stl = STL(serie_data_accidental_detrend, period=12)
res = stl.fit()

serie_data_accidental_saison = res.seasonal

# Affiche les résultats
f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(serie_data_accidental_detrend)
ax1.plot(serie_data_saison)
ax1.set_title("Série originale et saisonnalité")

ax2.plot(serie_data_accidental_detrend - serie_data_accidental_saison)
ax2.set_title("Série sans tendance ni saisonnalité")

In [ ]:
import statsmodels.api as sm

serie_test = serie_data_accidental
serie_test = serie_data_accidental_detrend
serie_test = serie_data_accidental_detrend - serie_data_accidental_saison

adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

In [ ]:
# ACF & PACF de la série

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_data_accidental_detrend - serie_data_accidental_saison, ax=ax1, lags = range(0,30))
ax1.set_title("Autocorrélation de la série")

plot_pacf(serie_data_accidental_detrend - serie_data_accidental_saison, ax=ax2, lags = range(0, 30))
ax2.set_title("Autocorrélation partielle de la série")